In [1]:
from nltk import bigrams,trigrams, download 
from nltk.corpus import reuters
from collections import Counter, defaultdict
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus

ModuleNotFoundError: No module named 'nltk'

In [ ]:
pathForWikiDataset = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
wikiSentences = WikiCorpus(pathForWikiDataset).get_texts()
download('punkt')
download('reuters')
reutersSentences  = reuters.sents()
print(reutersSentences)
print(wikiSentences)

In [ ]:
def calculateProbabilities(sentenceModel):
  for nextWord in sentenceModel:
    nextWords = sentenceModel[nextWord]
    total_Word_Count = float(sum(nextWords.values()))
    for previousWord in nextWords:
      nextWords[previousWord]/=total_Word_Count
    
def calculateSigleWordProbability(sentenceModel,wordCount):
  for word in sentenceModel:
    sentenceModel[word]/=wordCount

def convertToLower(pa):
  if type(pa)==str:
    return pa.lower()
  return pa

In [ ]:

sentenceModel4 = defaultdict(lambda: set())
sentenceModel5 = defaultdict(lambda: set())

def calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,sentences):
  wordCount = 0
  for sentence in sentences:
    #print(sentence)
    for word in sentence:
      wordCount+=1
      sentenceModel1[word]+=1
    for previousWord2,previousWord1,nextWord in trigrams(sentence,pad_right=True,pad_left=True):
      previousWord1 = convertToLower(previousWord1)
      previousWord2 = convertToLower(previousWord2)
      # print(previousWord1)
      # print(previousWord2)
      nextWord = convertToLower(nextWord)
      sentenceModel2[nextWord][previousWord1]+=1
      sentenceModel3[nextWord][previousWord2]+=1
      sentenceModel4[previousWord1].add(nextWord)
      sentenceModel5[previousWord2].add(nextWord)

  return wordCount

In [ ]:
sentenceModel1 = defaultdict(lambda:0)
sentenceModel2 = defaultdict(lambda: defaultdict(lambda:0))
sentenceModel3 = defaultdict(lambda: defaultdict(lambda:0))

In [ ]:
wikiWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,wikiSentences)
print(wikiWordCount)

In [ ]:
reutersWordCount = calculateWordCount(sentenceModel1,sentenceModel2,sentenceModel3,reutersSentences)
print(reutersWordCount)

In [ ]:
total_word = wikiWordCount + reutersWordCount
calculateSigleWordProbability(sentenceModel1,total_word)

In [ ]:
maxProbabilityWords = []
def makeWordSuggestionByTrigram(previousWord2,previousWord1):
  for nextWord in sentenceModel4[previousWord1] and sentenceModel5[previousWord2]:
    naiveBiasTrigramWeight = sentenceModel1[nextWord] * sentenceModel2[nextWord][previousWord1] * sentenceModel3[nextWord][previousWord2]
    maxProbabilityWords.append((nextWord,naiveBiasTrigramWeight))

In [ ]:
makeWordSuggestionByTrigram('my','name')
maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
print(*maxProbabilityWords[:10])

In [ ]:
while(True):
    text = input("Enter: ")
    if text == "stop":
        break
    try:
            maxProbabilityWords = []
            text = text.split(" ")
            makeWordSuggestionByTrigram(text[0],text[1])
            maxProbabilityWords.sort(key=lambda o:o[1],reverse=True)
            print(*maxProbabilityWords[:5])
            
        except:
            continue